In [ ]:
import numpy as np
import pandas as pd
import yfinace as yf
import matplotlib.pyplot as plt
import datetime

In [5]:
# import to ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout